Deep Learning
=============

Assignment 6
------------

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # number of letters available which are [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0]) # 97 which is a

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [43]:
batch_size=64 # number of phrases
num_unrollings=10 # number of letters in a phrase

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    
    np.shape(batches) == (num_unrollings + 1, batch_size, vocabulary_size)
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  print('batches[0]', batches[0].shape)
  s = [''] * batches[0].shape[0]
  for b in batches:
    print('current', characters(b))
    s = [''.join(x) for x in zip(s, characters(b))]
    print('accumulation', s)
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

batches[0] (64, 27)
current ['o', 'w', 'l', ' ', 'm', 'h', 'y', 'a', 't', 'm', 'n', 'h', 'e', 'e', 'o', 'y', 'o', 'a', ' ', 'a', 'i', ' ', 't', 'd', 'f', 'a', 'e', 'e', 'a', 'r', 'i', 'o', 'a', 'g', 'i', 'r', 'c', 'a', ' ', 'm', 't', 'u', 'e', 'o', 'o', 's', 'k', 'e', 'w', 'e', 't', 'e', ' ', 'i', 't', 'd', 't', 'e', 'f', 'd', 't', 'a', 'a', 's']
accumulation ['o', 'w', 'l', ' ', 'm', 'h', 'y', 'a', 't', 'm', 'n', 'h', 'e', 'e', 'o', 'y', 'o', 'a', ' ', 'a', 'i', ' ', 't', 'd', 'f', 'a', 'e', 'e', 'a', 'r', 'i', 'o', 'a', 'g', 'i', 'r', 'c', 'a', ' ', 'm', 't', 'u', 'e', 'o', 'o', 's', 'k', 'e', 'w', 'e', 't', 'e', ' ', 'i', 't', 'd', 't', 'e', 'f', 'd', 't', 'a', 'a', 's']
current ['n', 'h', 'l', 'a', 'a', 'e', ' ', 'y', 'i', 'i', 'e', 'e', ' ', 'b', ' ', 'e', 'r', ' ', 't', 'r', 't', 'a', 'i', 'y', ' ', 't', ' ', 'n', 'm', 'v', 'o', ' ', ' ', 'h', 'n', 'o', 'a', 's', 'd', 'o', ' ', ' ', ' ', ' ', 'f', ' ', 'l', 'r', 's', 't', 'h', 't', 's', 's', 'i', ' ', 'h', 'n', 'e', 'u', 'r', 't'

In [45]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [51]:
d = random_distribution()
print(d)
print(sample(d))


[[ 0.05438547  0.03965287  0.05743235  0.04292124  0.03198203  0.03071567
   0.04424208  0.05867551  0.01144918  0.01963431  0.03194882  0.0336723
   0.00728191  0.02742514  0.06646397  0.0579472   0.01331255  0.01210582
   0.06303618  0.02500562  0.06214351  0.01163936  0.05444383  0.01188161
   0.04564844  0.04375343  0.0411996 ]]
[[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.]]


Simple LSTM Model.

In [52]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Connect variables as a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [9]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.296785 learning rate: 10.000000
Minibatch perplexity: 27.03
f lbe ng prgrqa afnjranvatxhrabsmeevv   wgqxi t  pqtlyxdqqowzhhak byweabcxwwhooa
uqeutfxn  yoxzc ehzwgojggmovgsmcabo iakejr jyltiy w tlabos  m qyonpem t  ofijjuj
eowitsl rhe k widabtnopwlh  tl iwvmnl carocf d mde kxaaimpmutbttkponevmzlmlyl ti
mwe   omxd  mhbzrnstyns  cgg wli   poelalettiuwp  ooznnliugmhkjx noupxligt d pne
qs the qmwlsvcllv zz olcseaie f x dagmgrentegyagmh blseldxnnilsenxo uoze trsmdur
Validation set perplexity: 20.27
Average loss at step 100: 2.587147 learning rate: 10.000000
Minibatch perplexity: 11.24
Validation set perplexity: 10.66
Average loss at step 200: 2.238520 learning rate: 10.000000
Minibatch perplexity: 8.47
Validation set perplexity: 8.68
Average loss at step 300: 2.094368 learning rate: 10.000000
Minibatch perplexity: 7.35
Validation set perplexity: 8.12
Average loss at step 400: 1.996569 learning rate: 10.000000
Minibatch perplexity: 7.72
Validation set per

Validation set perplexity: 4.39
Average loss at step 4500: 1.614420 learning rate: 10.000000
Minibatch perplexity: 5.05
Validation set perplexity: 4.61
Average loss at step 4600: 1.614374 learning rate: 10.000000
Minibatch perplexity: 4.98
Validation set perplexity: 4.56
Average loss at step 4700: 1.626260 learning rate: 10.000000
Minibatch perplexity: 5.13
Validation set perplexity: 4.47
Average loss at step 4800: 1.629644 learning rate: 10.000000
Minibatch perplexity: 4.42
Validation set perplexity: 4.44
Average loss at step 4900: 1.633215 learning rate: 10.000000
Minibatch perplexity: 5.09
Validation set perplexity: 4.58
Average loss at step 5000: 1.604255 learning rate: 1.000000
Minibatch perplexity: 4.52
gn been three nine eyi is piles and persically to denomes and diangamend in aran
feque interval is one kwanned the nostinus of distinations simble a micone wince
 of whoside kind stills and or diepe fron sestines non of the world is langustyd
nepe clain he imponeds de youncel exta

In [62]:
num_nodes = 32

graph = tf.Graph()
with graph.as_default():
  
  # Parameters: [ Input Forget Memory Output ]
  px = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes * 4], -0.1, 0.1)) # for input
  pm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1)) # for previous output
  pb = tf.Variable(tf.zeros([1, num_nodes * 4])) # for bias

  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Connect variables as a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    vals = tf.matmul(i, px) + tf.matmul(o, pm) + pb

    input_gate = tf.sigmoid(vals[:, :num_nodes])
    forget_gate = tf.sigmoid(vals[:, num_nodes:num_nodes*2])
    update = vals[:, num_nodes*2:num_nodes*3]
    output_gate = tf.sigmoid(vals[:, num_nodes*3:num_nodes*4])

    state = forget_gate * state + input_gate * tf.tanh(update)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [63]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.293678 learning rate: 10.000000
Minibatch perplexity: 26.94
lonhwhpjjulcsexhrpwy jssqmca dv  rioszxi  fznrm oaudg  tstsefljt fyxs  vzeqivnnq
yeqnwbgl x  gl mythbtevk aesrpasvhsk s  dyehtdy vee  d f aenymsfpsxl tlwrepcrm  
y nkyft xbheuekswxwkkefqmroap t sib  zmsndcrjdkrbzk thziexzzrt z f pv ct tqsswq 
tnr rbtcke  ritgts tayo osyia zfa    st d vifu tykyaqz ir bht jacedmaitcg zoaisp
zuidhaaeouhsiyaesf  wvyegkjuw rvw lreg gitnizmhytnw mrueiezrnoh  oh ree  xttleao
Validation set perplexity: 20.27
Average loss at step 100: 2.631478 learning rate: 10.000000
Minibatch perplexity: 10.81
Validation set perplexity: 10.56
Average loss at step 200: 2.266104 learning rate: 10.000000
Minibatch perplexity: 8.60
Validation set perplexity: 9.37
Average loss at step 300: 2.120927 learning rate: 10.000000
Minibatch perplexity: 6.93
Validation set perplexity: 8.44
Average loss at step 400: 2.070373 learning rate: 10.000000
Minibatch perplexity: 8.01
Validation set per

Validation set perplexity: 5.75
Average loss at step 4500: 1.787090 learning rate: 10.000000
Minibatch perplexity: 6.04
Validation set perplexity: 5.89
Average loss at step 4600: 1.773914 learning rate: 10.000000
Minibatch perplexity: 6.52
Validation set perplexity: 5.82
Average loss at step 4700: 1.770067 learning rate: 10.000000
Minibatch perplexity: 5.59
Validation set perplexity: 5.77
Average loss at step 4800: 1.764159 learning rate: 10.000000
Minibatch perplexity: 5.71
Validation set perplexity: 5.78
Average loss at step 4900: 1.766486 learning rate: 10.000000
Minibatch perplexity: 5.93
Validation set perplexity: 5.74
Average loss at step 5000: 1.759311 learning rate: 1.000000
Minibatch perplexity: 5.47
hunes areainid curdscemon over the habling brough there one nine sex disprophin 
quiojers to llectical concects aughend other at bonarraty and apacution malled s
ers aterlyle pertol arcombesk of hissoal in surdper bra mides con the lengy cond
 jines in are reproled ham on the ant 